In [2]:
import pandas as pd
import sqlite3


In [3]:
df1 = pd.read_csv("credit-card-approval-prediction/application_record.csv")
df2 = pd.read_csv("credit-card-approval-prediction/credit_record.csv")


In [4]:
df1.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [5]:
df2.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [6]:
conn = sqlite3.connect(":memory:")

In [7]:
df1.to_sql("application_record", conn, index=False, if_exists="replace")
df2.to_sql("credit_record", conn, index=False, if_exists="replace")


1048575

In [8]:
query = """
SELECT *
FROM application_record AS ar
JOIN credit_record AS cr ON ar.ID = cr.ID
"""
result = pd.read_sql_query(query, conn)


In [9]:
result.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ID,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,None,2.0,5008804,-15,X
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,None,2.0,5008804,-14,0
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,None,2.0,5008804,-13,1
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,None,2.0,5008804,-12,C
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,None,2.0,5008804,-11,C


Average Income by Gender

In [10]:
query = """
SELECT CODE_GENDER AS gender, ROUND(AVG(AMT_INCOME_TOTAL), 2) AS avg_income
FROM application_record
GROUP BY CODE_GENDER
"""
avg_income_by_gender = pd.read_sql_query(query, conn)
avg_income_by_gender

,gender,avg_income
0,F,174523.04
1,M,214086.64


**Status:
0: 1-29 days past due 
1: 30-59 days past due 
2: 60-89 days overdue 
3: 90-119 days overdue 
4: 120-149 days overdue 
5: Overdue or bad debts, write-offs for more than 150 days 
C: paid off that month 
X: No loan for the month**

Count of Customers by Credit Status:

In [11]:
query = """
SELECT STATUS AS status, COUNT(DISTINCT ID) AS customer_count
FROM credit_record
GROUP BY STATUS
ORDER BY customer_count
"""
status_counts = pd.read_sql_query(query, conn)
status_counts

,status,customer_count
0,4,190
1,5,195
2,3,268
3,2,597
4,1,5201
5,C,22707
6,X,25005
7,0,39980


 Monthly Trend of Credit Status Changes - This Month:

In [12]:
query_current_month = """
SELECT cr.MONTHS_BALANCE AS this_month, cr.STATUS AS status, COUNT(cr.ID) AS count_customers
FROM credit_record AS cr
WHERE cr.MONTHS_BALANCE = 0
GROUP BY cr.STATUS
ORDER BY count_customers DESC
"""
current_month_trend = pd.read_sql_query(query_current_month, conn)
current_month_trend


,this_month,status,count_customers
0,0,C,17613
1,0,0,8914
2,0,X,6925
3,0,1,309
4,0,5,65
5,0,2,19
6,0,3,6
7,0,4,5


 Monthly Trend of Credit Status Changes - Last Month:

In [13]:
query_previous_month = """
SELECT cr.MONTHS_BALANCE AS last_month, cr.STATUS AS status, COUNT(cr.ID) AS count_customers
FROM credit_record AS cr
WHERE cr.MONTHS_BALANCE = -1
GROUP BY cr.STATUS
ORDER BY count_customers DESC
"""
previous_month_trend = pd.read_sql_query(query_previous_month, conn)
previous_month_trend


,last_month,status,count_customers
0,-1,C,17504
1,-1,0,10301
2,-1,X,6163
3,-1,1,372
4,-1,5,62
5,-1,2,17
6,-1,4,11
7,-1,3,6


 Monthly Trend of Credit Status Changes - Two Month Ago:

In [14]:
query_two_months_ago = """
SELECT cr.MONTHS_BALANCE, cr.STATUS, COUNT(cr.ID) AS count_customers
FROM credit_record AS cr
WHERE cr.MONTHS_BALANCE = -2
GROUP BY cr.STATUS
ORDER BY count_customers DESC
"""
two_months_ago_trend = pd.read_sql_query(query_two_months_ago, conn)
two_months_ago_trend


,MONTHS_BALANCE,STATUS,count_customers
0,-2,C,17139
1,-2,0,10398
2,-2,X,6215
3,-2,1,367
4,-2,5,57
5,-2,2,19
6,-2,3,11
7,-2,4,3


Default Rate As A Percentage By Employment Status:

In [15]:
query_default_rate_percentage = """
SELECT 
    CASE 
        WHEN ar.DAYS_EMPLOYED > 0 THEN 'Unemployed'
        ELSE 'Employed'
    END AS employment_status,
    COUNT(CASE WHEN cr.STATUS IN ('1', '2', '3', '4', '5') THEN 1 END) AS count_defaults,
    COUNT(*) AS total_customers,
    ROUND(((COUNT(CASE WHEN cr.STATUS IN ('1', '2', '3', '4', '5') THEN 1 END) * 1.0 / COUNT(*)) * 100), 2) AS default_rate_percentage
FROM application_record AS ar
LEFT JOIN credit_record AS cr ON ar.ID = cr.ID
GROUP BY employment_status
"""

# Execute the query
default_rate_percentage = pd.read_sql_query(query_default_rate_percentage, conn)
default_rate_percentage

,employment_status,count_defaults,total_customers,default_rate_percentage
0,Employed,9922,982649,1.01
1,Unemployed,1653,197166,0.84


In [16]:
query = """
SELECT 
    CASE 
        WHEN ar.AMT_INCOME_TOTAL <= 20000 THEN 'Low Income'
        WHEN ar.AMT_INCOME_TOTAL <= 50000 THEN 'Medium Income'
        WHEN ar.AMT_INCOME_TOTAL <= 100000 THEN 'High Income'
        ELSE 'Very High Income'
    END AS income_bracket,
    AVG(CASE WHEN cr.STATUS IN ('1', '2', '3', '4', '5') THEN 1 ELSE 0 END) AS default_rate
FROM application_record AS ar
LEFT JOIN credit_record AS cr ON ar.ID = cr.ID
GROUP BY income_bracket
ORDER BY income_bracket
"""
income_risk = pd.read_sql_query(query, conn)
income_risk


,income_bracket,default_rate
0,High Income,0.009851
1,Medium Income,0.006932
2,Very High Income,0.009831


In [17]:
query = """
SELECT 
    cr.MONTHS_BALANCE,
    CASE 
        WHEN ar.AMT_INCOME_TOTAL <= 20000 THEN 'Low Income'
        WHEN ar.AMT_INCOME_TOTAL <= 50000 THEN 'Medium Income'
        WHEN ar.AMT_INCOME_TOTAL <= 100000 THEN 'High Income'
        ELSE 'Very High Income'
    END AS income_bracket,
    AVG(CASE WHEN cr.STATUS IN ('1', '2', '3', '4', '5') THEN 1 ELSE 0 END) AS default_rate
FROM application_record AS ar
LEFT JOIN credit_record AS cr ON ar.ID = cr.ID
WHERE cr.MONTHS_BALANCE >= -2 
GROUP BY cr.MONTHS_BALANCE, income_bracket
ORDER BY cr.MONTHS_BALANCE, income_bracket
"""

monthly_risk = pd.read_sql_query(query, conn)
monthly_risk


,MONTHS_BALANCE,income_bracket,default_rate
0,-2,High Income,0.015789
1,-2,Medium Income,0.004695
2,-2,Very High Income,0.014327
3,-1,High Income,0.014408
4,-1,Medium Income,0.013761
5,-1,Very High Income,0.014383
6,0,High Income,0.012604
7,0,Medium Income,0.013699
8,0,Very High Income,0.013255


In [20]:
query = """
SELECT 
    ar.NAME_FAMILY_STATUS as family_status,
    CASE 
        WHEN ar.CNT_CHILDREN = 0 THEN 'No Children'
        WHEN ar.CNT_CHILDREN BETWEEN 1 AND 2 THEN '1-2 Children'
        ELSE '3+ Children'
    END AS children_bracket,
    AVG(CASE WHEN cr.STATUS IN ('1', '2', '3', '4', '5') THEN 1 ELSE 0 END) AS default_rate
FROM application_record AS ar
LEFT JOIN credit_record AS cr ON ar.ID = cr.ID
GROUP BY ar.NAME_FAMILY_STATUS, children_bracket
HAVING COUNT(*) > 10  -- Optional: Ensure each group has enough data
ORDER BY default_rate DESC
"""
family_risk = pd.read_sql_query(query, conn)
family_risk


,family_status,children_bracket,default_rate
0,Single / not married,3+ Children,0.029412
1,Single / not married,1-2 Children,0.016900
2,Separated,3+ Children,0.011516
3,Married,3+ Children,0.011324
4,Single / not married,No Children,0.011002
5,Married,1-2 Children,0.010170
6,Separated,No Children,0.009555
7,Civil marriage,No Children,0.009476
8,Widow,No Children,0.009442
9,Civil marriage,1-2 Children,0.009437


In [21]:
query = """
SELECT ar.OCCUPATION_TYPE,
       AVG(CASE WHEN cr.STATUS IN ('1', '2', '3', '4', '5') THEN 1 ELSE 0 END) AS default_rate
FROM application_record AS ar
LEFT JOIN credit_record AS cr ON ar.ID = cr.ID
GROUP BY ar.OCCUPATION_TYPE
ORDER BY default_rate DESC
LIMIT 10
"""
occupation_risk = pd.read_sql_query(query, conn)
occupation_risk


,OCCUPATION_TYPE,default_rate
0,IT staff,0.024155
1,Low-skill Laborers,0.021654
2,HR staff,0.016421
3,Security staff,0.016413
4,Waiters/barmen staff,0.015069
5,High skill tech staff,0.012627
6,Realty agents,0.012601
7,Medicine staff,0.011178
8,Managers,0.010687
9,Drivers,0.010498


In [24]:
query = """
WITH CurrentMonthDefaults AS (
    SELECT 
        ar.ID,
        ar.AMT_INCOME_TOTAL,
        ar.NAME_FAMILY_STATUS,
        ar.NAME_INCOME_TYPE,
        ar.NAME_HOUSING_TYPE,
        ar.OCCUPATION_TYPE,
        CASE 
            WHEN ar.DAYS_EMPLOYED > 0 THEN 'Unemployed'
            ELSE 'Employed'
        END AS employment_status,
        cr.STATUS
    FROM application_record AS ar
    LEFT JOIN credit_record AS cr ON ar.ID = cr.ID
    WHERE cr.MONTHS_BALANCE = 0  -- Current month
)

SELECT 
    employment_status,
    NAME_FAMILY_STATUS AS family_status,
    NAME_INCOME_TYPE AS income,
    NAME_HOUSING_TYPE AS housing_type,
    OCCUPATION_TYPE AS occupation,
    CASE 
        WHEN AMT_INCOME_TOTAL <= 20000 THEN 'Low Income'
        WHEN AMT_INCOME_TOTAL <= 50000 THEN 'Medium Income'
        WHEN AMT_INCOME_TOTAL <= 100000 THEN 'High Income'
        ELSE 'Very High Income'
    END AS income_bracket,
    AVG(CASE WHEN STATUS IN ('1', '2', '3', '4', '5') THEN 1 ELSE 0 END) AS default_rate
FROM CurrentMonthDefaults
GROUP BY employment_status, NAME_FAMILY_STATUS, NAME_INCOME_TYPE, NAME_HOUSING_TYPE, OCCUPATION_TYPE, income_bracket
ORDER BY default_rate DESC
LIMIT 10
"""

default_drivers_current_month = pd.read_sql_query(query, conn)
default_drivers_current_month


,employment_status,family_status,income,housing_type,occupation,income_bracket,default_rate
0,Employed,Married,Pensioner,House / apartment,None,High Income,1.000000
1,Employed,Married,Pensioner,With parents,None,Very High Income,1.000000
2,Employed,Separated,Commercial associate,House / apartment,Medicine staff,Very High Income,1.000000
3,Employed,Separated,Working,Municipal apartment,Managers,Very High Income,1.000000
4,Employed,Single / not married,Commercial associate,Municipal apartment,Cooking staff,Very High Income,1.000000
5,Employed,Single / not married,Pensioner,House / apartment,Low-skill Laborers,Very High Income,1.000000
6,Employed,Single / not married,Working,Rented apartment,Cleaning staff,High Income,1.000000
7,Employed,Widow,Working,Municipal apartment,Security staff,Very High Income,0.666667
8,Employed,Married,Pensioner,House / apartment,Core staff,Very High Income,0.500000
9,Employed,Separated,Commercial associate,House / apartment,Core staff,High Income,0.500000
